In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("data_processed.csv")
df

,Giá,Năm sản xuất,Kiểu dáng,Xuất xứ,Số km đã đi,Hộp số,Nhiên liệu
0,1.119000e+09,2016,Sedan,Trongnước,51.499,Sốtựđộng,Xăng
1,1.789000e+09,2021,SUV,Nhậpkhẩu,8.000,Sốtựđộng,Xăng
2,5.200000e+08,2021,MPV,Nhậpkhẩu,10.000,Sốtay,Xăng
3,3.200000e+08,2009,Sedan,Trongnước,90.000,Sốtựđộng,Xăng
4,7.300000e+08,2016,SUV,Trongnước,68.000,Sốtựđộng,Xăng
...,...,...,...,...,...,...,...
2291,6.420000e+08,2018,Pick-upTruck,Nhậpkhẩu,15.000,Sốtựđộng,Diesel
2292,8.980000e+08,2018,Sedan,Trongnước,50.000,Sốtựđộng,Xăng
2293,3.800000e+08,2014,Hatchback,Nhậpkhẩu,92.000,Sốtựđộng,Xăng
2294,1.270000e+08,1993,Sedan,Nhậpkhẩu,68.000,Sốtựđộng,Xăng


# 1. Linear Regression: Đơn giản và dễ hiểu, thích hợp cho dữ liệu tuyến tính.

In [22]:
 # Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float) 

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=60
)

# Huấn luyện mô hình
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "Sedan",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()  
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))  # Tạo DataFrame từ mảng NumPy


y_pred = model.predict(X_test)

# Tính các chỉ số đánh giá
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)  # Kết hợp DataFrame

# Dự đoán giá xe
predicted_price = model.predict(new_car_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Mean Absolute Error (MAE): 591140337.724638
Mean Squared Error (MSE): 9.167266394475543e+17
R² Score: 0.2633008020622033
Giá xe dự đoán: 1,090,725,848 VND


Kết quả của mô hình dự đoán giá xe sử dụng Linear Regression như sau:

- **Mean Absolute Error (MAE)**: Khoảng 591,140,337.72 VND. Điều này có nghĩa là trung bình các giá trị tuyệt đối của sự sai lệch giữa giá trị dự đoán và giá trị thực tế là khoảng 591 triệu VND.
  
- **Mean Squared Error (MSE)**: Khoảng 9.167266394475543e+17 VND². Đây là trung bình của bình phương của sự sai lệch giữa giá trị dự đoán và giá trị thực tế.
  
- **R² Score**: Khoảng 0.26. Đây là một chỉ số đánh giá khả năng giải thích của mô hình, dao động từ 0 đến 1. Giá trị càng gần 1 thể hiện một mô hình tốt hơn. Trong trường hợp này, R² Score chỉ là khoảng 0.26, cho thấy mô hình không thực sự tốt trong việc giải thích biến động của dữ liệu.

- **Giá xe dự đoán**: Khoảng 1,090,725,848 VND. Đây là giá dự đoán của mô hình cho dữ liệu mới.

Dựa trên các kết quả này, có thể thấy rằng mô hình Linear Regression không đạt được hiệu suất tốt trong việc dự đoán giá xe. MAE và MSE đều rất cao, và R² Score thấp, chỉ 26%. Điều này cho thấy mô hình không thực sự phù hợp với dữ liệu và có thể cần phải cải thiện hoặc thay đổi mô hình để đạt được dự đoán chính xác hơn.

# Ridge Regression

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Ridge Regression
model = Ridge(alpha=1.0)  # Alpha là hyperparameter điều chỉnh độ phức tạp

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán giá trị cho tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá hiệu suất của mô hình
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

Mean Squared Error: 2.232132593422841e+18
Root Mean Squared Error: 1494032326.766339
R-squared: 0.20045134909215012


# Sử dụng GridSearchCV cho Ridge Regression

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.1, random_state=90
)

# Tạo mô hình Ridge Regression
ridge_model = Ridge()

# Grid search để tìm alpha tốt nhất
param_grid = {'alpha': [0.1, 1, 10,100,200]}
grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Lấy mô hình tốt nhất
best_ridge_model = grid_search.best_estimator_

# Đánh giá mô hình tốt nhất trên tập kiểm tra
y_pred = best_ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Best Ridge Model - Mean Squared Error (MSE):", mse)
print("Best Ridge Model - R² Score:", r2)

Best Ridge Model - Mean Squared Error (MSE): 5.622672485666686e+17
Best Ridge Model - R² Score: 0.4158102315105834


Kết quả trông tốt hơn sau khi thực hiện GridSearchCV để tìm giá trị alpha tốt nhất cho mô hình Ridge Regression. MSE đã giảm và R² Score đã tăng tuy nhiên kết quả vẫn còn rất thấp, sử dụng các mô hình khác để dự đoán

# Gradient Boosting Regression

In [6]:
 from sklearn.ensemble import GradientBoostingRegressor
 # Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=0
)

# Huấn luyện mô hình
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))  # Tạo DataFrame từ mảng NumPy


y_pred = model.predict(X_test)

# Tính các chỉ số đánh giá
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)  # Kết hợp DataFrame

# Dự đoán giá xe
predicted_price = model.predict(new_car_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Mean Absolute Error (MAE): 446646686.96598685
Mean Squared Error (MSE): 1.120954575786552e+18
R² Score: 0.4545526020712197
Giá xe dự đoán: 1,151,198,215 VND


- Mean Absolute Error (MAE): 446,646,686.97 VND, đây là trung bình tổng các giá trị tuyệt đối của sự sai lệch giữa giá trị dự đoán và giá trị thực tế.
- Mean Squared Error (MSE): 1.1209545757865522e+18, đây là trung bình tổng bình phương của sự sai lệch giữa giá trị dự đoán và giá trị thực tế.
- R² Score: 0.4545, đây là một chỉ số đo lường khả năng giải thích của mô hình, dao động từ 0 đến 1. Giá trị càng gần 1 thể hiện một mô hình tốt hơn.
- Độ chính xác của mô hình: 45.46%, đây là tỷ lệ phần trăm của sự biến động trong biến số phụ thuộc mà mô hình có thể giải thích.
- Giá xe dự đoán: 1,141,295,004 VND, đây là giá trị dự đoán của mô hình cho dữ liệu mới.

# test

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả và chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "").astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo danh sách mô hình và tham số
models = [
    {
        'name': 'Gradient Boosting Regression',
        'model': GradientBoostingRegressor(),
        'param_distributions': {
            'n_estimators': [50, 100, 200, 300],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'subsample': [0.6, 0.8, 1.0],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    {
        'name': 'Random Forest Regression',
        'model': RandomForestRegressor(),
        'param_distributions': {
            'n_estimators': [50, 100, 200, 300],
            'max_depth': [5, 10, 15, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2']
        }
    }
]

# Lặp qua từng mô hình và tối ưu
for model_info in models:
    print(f"Huấn luyện mô hình: {model_info['name']}")
    
    # Khởi tạo RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=model_info['model'],
        param_distributions=model_info['param_distributions'],
        n_iter=50,  # Số lần lặp
        cv=5,
        scoring='neg_mean_squared_error',
        random_state=42,
        n_jobs=-1  # Sử dụng tất cả các core CPU để tăng tốc
    )

    # Huấn luyện RandomizedSearchCV với dữ liệu train
    random_search.fit(X_train, y_train)

    # In ra hyperparameter tốt nhất
    print(f"Hyperparameter tốt nhất cho {model_info['name']}: {random_search.best_params_}")

    # Lấy mô hình tốt nhất
    best_model = random_search.best_estimator_

    # Dự đoán giá trị cho tập kiểm tra
    y_pred = best_model.predict(X_test)

    # Đánh giá hiệu suất của mô hình tốt nhất
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"{model_info['name']} - R² Score:", r2)
    print(f"{model_info['name']} - Mean Absolute Error (MAE):", mae)
    print(f"{model_info['name']} - Mean Squared Error (MSE):", mse)
    print("-" * 50)

Huấn luyện mô hình: Gradient Boosting Regression
Hyperparameter tốt nhất cho Gradient Boosting Regression: {'subsample': 1.0, 'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 7, 'learning_rate': 0.1}
Gradient Boosting Regression - R² Score: 0.2976591763774814
Gradient Boosting Regression - Mean Absolute Error (MAE): 418796169.0843625
Gradient Boosting Regression - Mean Squared Error (MSE): 1.9607535355343255e+18
--------------------------------------------------
Huấn luyện mô hình: Random Forest Regression
Hyperparameter tốt nhất cho Random Forest Regression: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 15}
Random Forest Regression - R² Score: 0.3810148860735898
Random Forest Regression - Mean Absolute Error (MAE): 384692399.507012
Random Forest Regression - Mean Squared Error (MSE): 1.728045999539721e+18
--------------------------------------------------


# Nhận xét sau khi train 2 model:
R² Score: Cả hai mô hình đều cho kết quả R² Score khá thấp, cho thấy chúng không giải thích được nhiều biến thiên của giá xe.
Gradient Boosting: R² Score = 0.2976 (29.76%)
Random Forest: R² Score = 0.3810 (38.10%)
MAE: Lỗi tuyệt đối trung bình của Random Forest thấp hơn so với Gradient Boosting, nghĩa là dự đoán của Random Forest có xu hướng gần với giá trị thực tế hơn.
MSE: Lỗi bình phương trung bình của Random Forest cũng thấp hơn Gradient Boosting.
Kết luận:
Random Forest hiệu quả hơn: Trong trường hợp này, Random Forest cho kết quả tốt hơn Gradient Boosting, mặc dù cả hai mô hình đều chưa thực sự hiệu quả.
Cần cải thiện mô hình: R² Score thấp cho thấy mô hình vẫn chưa đủ tốt để dự đoán giá xe một cách chính xác.nh xác.

# Mô hình Random Forest Regression

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float) 

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=60
)

# Huấn luyện mô hình Random Forest
model = RandomForestRegressor(n_estimators=80, random_state=0)  # Thiết lập số cây quyết định là 100
model.fit(X_train, y_train)

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "Sedan",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()  
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))  # Tạo DataFrame từ mảng NumPy

new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)  # Kết hợp DataFrame

# Dự đoán giá xe
predicted_price = model.predict(new_car_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")

# Đánh giá mô hình
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)


Giá xe dự đoán: 1,139,737,500 VND
Mean Absolute Error (MAE): 338912389.28611445
Mean Squared Error (MSE): 7.587635445252539e+17
R² Score: 0.3902429899789408


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả và chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "").astype(float)

# Feature Engineering: Tạo thêm thuộc tính 'Tuổi xe'
current_year = 2023
df['Tuổi xe'] = current_year - df['Năm sản xuất']

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi", 'Tuổi xe']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Random Forest Regression
model = RandomForestRegressor()

# Định nghĩa các giá trị hyperparameter cần thử nghiệm
param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# In ra hyperparameter tốt nhất
print("Hyperparameter tốt nhất:", random_search.best_params_)

# Lấy mô hình tốt nhất
best_model = random_search.best_estimator_

# Dự đoán giá trị cho tập kiểm tra
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Random Forest - R² Score:", r2)
print("Random Forest - Mean Absolute Error (MAE):", mae)
print("Random Forest - Mean Squared Error (MSE):", mse)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data['Tuổi xe'] = current_year - new_data['Năm sản xuất']  # Tạo thuộc tính 'Tuổi xe'
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price = best_model.predict(new_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price, "VND")

Hyperparameter tốt nhất: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10}
Random Forest - R² Score: 0.39431672814617524
Random Forest - Mean Absolute Error (MAE): 396539906.26692295
Random Forest - Mean Squared Error (MSE): 1.6909107042589796e+18
Giá xe dự đoán: 1,483,627,785 VND


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả và chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "").astype(float)

# Feature Engineering: Tạo thêm thuộc tính 'Tuổi xe'
current_year = 2023
df['Tuổi xe'] = current_year - df['Năm sản xuất']

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi", 'Tuổi xe']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Random Forest Regression
model = RandomForestRegressor()

# Định nghĩa các giá trị hyperparameter cần thử nghiệm
param_distributions = {
    'n_estimators': [ 200, 300, 700],
    'max_depth': [10, 15, 20, 25, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# In ra hyperparameter tốt nhất
print("Hyperparameter tốt nhất:", random_search.best_params_)

# Lấy mô hình tốt nhất
best_model = random_search.best_estimator_

# Dự đoán giá trị cho tập kiểm tra
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Random Forest - R² Score:", r2)
print("Random Forest - Mean Absolute Error (MAE):", mae)
print("Random Forest - Mean Squared Error (MSE):", mse)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data['Tuổi xe'] = current_year - new_data['Năm sản xuất']  # Tạo thuộc tính 'Tuổi xe'
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price = best_model.predict(new_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price, "VND")

Hyperparameter tốt nhất: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Random Forest - R² Score: 0.37081721512315624
Random Forest - Mean Absolute Error (MAE): 391669928.937627
Random Forest - Mean Squared Error (MSE): 1.7565152536365396e+18
Giá xe dự đoán: 1,343,933,707 VND


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Random Forest
model = RandomForestRegressor()

# Định nghĩa các giá trị hyperparameter cần thử nghiệm 
param_distributions = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=10,  # Số lần lặp
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# Lấy tổ hợp hyperparameter tốt nhất
best_params = random_search.best_params_

# Tạo mô hình mới với hyperparameter tốt nhất
best_model = RandomForestRegressor(**best_params)

# Huấn luyện mô hình với hyperparameter tốt nhất
best_model.fit(X_train, y_train)

# Dự đoán giá trị cho tập kiểm tra
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred)
mae_best = mean_absolute_error(y_test, y_pred)
mse_best = mean_squared_error(y_test, y_pred)

print("Random Forest (best) - R² Score:", r2_best)
print("Random Forest (best) - Mean Absolute Error (MAE):", mae_best)
print("Random Forest (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price_best,"VND")

Random Forest (best) - R² Score: 0.4124051690824463
Random Forest (best) - Mean Absolute Error (MAE): 382294367.72496617
Random Forest (best) - Mean Squared Error (MSE): 1.6404124656187046e+18
Giá xe dự đoán: 1,256,514,454 VND


# Tối ưu hóa bằng cách sử dụng kỹ thuật RandomizedSearchCV để tìm tổ hợp các hyperparameter

Các hyperparameter được thử nghiệm là:

n_estimators: Số lượng cây quyết định trong rừng. Các giá trị thử nghiệm là [50, 100, 200, 300, 400].

max_depth: Độ sâu tối đa của cây. Các giá trị thử nghiệm là [5, 10, 15, 20, 25].

min_samples_split: Số lượng mẫu tối thiểu cần có trong một nút để nút đó được phân chia thành các nút con. Các giá trị thử nghiệm là [2, 5, 10].

min_samples_leaf: Số lượng mẫu tối thiểu cần có trong một nút lá. Các giá trị thử nghiệm là [1, 2, 4].

max_features: Số lượng đặc trưng tối đa được xem xét khi tìm kiếm điểm tách tốt nhất. Các giá trị thử nghiệm là ['sqrt', 'log2'], trong đó 'sqrt' có nghĩa là sử dụng căn bậc hai của số lượng đặc trưng, và 'log2' có nghĩa là sử dụng logarit bằng 2 của số lượng đặc trưng.

RandomizedSearchCV sẽ thử nghiệm tất cả các tổ hợp khả dĩ của các giá trị hyperparameter này trong số lần lặp (n_iter=10) và sử dụng phương pháp cross-validation (cv=5) để đánh giá hiệu suất của mô hình với mỗi tổ hợp hyperparameter. Mô hình tốt nhất được chọn dựa trên giá trị neg_mean_squared_error thấp nhất (tương đương với giá trị MSE thấp nhất).
Sau khi tìm được tổ hợp hyperparameter tốt nhất, đoạn mã tạo một mô hình Random Forest mới với các hyperparameter đó, huấn luyện mô hình trên tập huấn luyện và đánh giá hiệu suất của mô hình trên tập kiểm tra bằng các chỉ số R² Score, MAE và MSE.

# Đánh giá:

Sau nhiều lần thử nghiệm nhiều mô hình và kỹ thuật tối ưu hóa khác nhau, có thể rút ra một số nhận xét về hiệu suất của mô hình dự đoán giá xe dựa trên tập dữ liệu

1. Hiệu suất mô hình chưa cao
- R² Score thấp: Mặc dù đã áp dụng Feature Engineering, Feature Selection, Log Transformation và RandomizedSearchCV để tối ưu hóa hyperparameter, R² Score của các mô hình vẫn ở mức thấp (khoảng 0.3 - 0.4). Điều này cho thấy mô hình vẫn chưa thể giải thích tốt biến thiên của giá xe.- 
MAE và MSE cao: MAE và MSE cũng ở mức khá cao, cho thấy dự đoán của mô hình vẫn còn sai lệch đáng kể so với giá trị thực tế

2. Nguyên nhân tiềm ẩn- 
Chất lượng dữ liệu: Rất có thể chất lượng dữ liệu là nguyên nhân chính dẫn đến hiệu suất thấp của mô hình
 .- 
Thuộc tính không đầy đủ: Có thể tập dữ liệu thiếu các thuộc tính quan trọng ảnh hưởng đến giá xe, ví dụ: tình trạng động cơ, hộp số, màu sơn, trang bị, lịch sử bảo dưỡ..
- .
Số lượng mẫu hạn chế: Số lượng mẫu trong tập dữ liệu có thể quá ít để mô hình học được các quy luật phức .ìa bạn..